## Outline

This notebook combines previous examples. Scaling the ingestion with casting rules per columns.

## Create sample table

In [2]:
\l ../../qparquet.q
p)import numpy as np

In [3]:
p)times=[np.datetime64('2012-06-30T21:00:00.000000000-0400')] * 4
p)table=pandas.DataFrame(columns=['time','one'])
p)table['time'] = times
p)table['one'] = [1,2,3,4]
p)print(table)

                 time  one
0 2012-07-01 01:00:00    1
1 2012-07-01 01:00:00    2
2 2012-07-01 01:00:00    3
3 2012-07-01 01:00:00    4


In [4]:
p)parquet.write_table(pyarrow.Table.from_pandas(table), 'example.parquet')

### convert.q

This script coordinates distributing the work of converting the parquet file across multiple processes

```q
//Load needed functions
\l ../../qparquet.q

//Open handles to worker processes
.z.pd:`u#asc hopen each"J"$(.Q.opt .z.X)`slaves

file:`:example.parquet;

columns:.qparquet.getColumnNames[file];

destination:`:splayed

pyConversions:enlist[`time]!enlist "table[\"time\"]=pandas.to_numeric(table[\"time\"])";
qConversions:enlist[`time]!enlist {`timestamp$x-`long$2000.01.01D-1970.01.01D};

start:.z.p;
//Distribute tasks to workers
//Each worker reads a column at a time
{[f;d;p;q;c]
  show string[.z.p],"  ",string c;
  .Q.dd[d;c] set .qparquet.getColumnsCustom[f;enlist c;p;q]c
    }[file;destination;pyConversions;qConversions] peach columns

//Add a .d file to the destination to inform q of the order of columns
.Q.dd[destination;`.d] set columns

end:.z.p;

show "Took ",string end-start;

//Load the converted table
\l splayed

//Query the q table
show select from splayed

```

## Running the example

Start your worker processes

```q
q ../../qparquet.q -p 5001 &
q ../../qparquet.q -p 5002 &
q ../../qparquet.q -p 5003 &
```

Run the master process to distibute the work

```bash
q convert.q -s -3 -slaves 5001 5002 5003
```

The output shows that the qparquet data is now successfully a q splayed table

```q
`:splayed/time`:splayed/one
`:splayed/.d
"Took 0D00:00:00.021990000"
`splayed
time                          one
---------------------------------
2012.07.01D01:00:00.000000000 1
2012.07.01D01:00:00.000000000 2
2012.07.01D01:00:00.000000000 3
2012.07.01D01:00:00.000000000 4
```